In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import re
import time
import tensorflow_datasets as tfds
import tensorflow as tf



import os
%cd /content/drive/MyDrive/사이드프로젝트 /ChatBot
print('현재 경로: ', os.getcwd())

/content/drive/MyDrive/사이드프로젝트 /ChatBot
현재 경로:  /content/drive/MyDrive/사이드프로젝트 /ChatBot


In [ ]:
train= pd.read_excel("data/Training.xlsx")
val=pd.read_excel('data/Validation.xlsx')

In [ ]:
df=pd.concat([train,val],ignore_index=True)
df.head()

,Unnamed: 0,연령,성별,상황키워드,신체질환,감정_대분류,감정_소분류,사람문장1,시스템문장1,사람문장2,시스템문장2,사람문장3,시스템문장3
0,1,청년,여성,"진로,취업,직장",해당없음,분노,노여워하는,일은 왜 해도 해도 끝이 없을까? 화가 난다.,많이 힘드시겠어요. 주위에 의논할 상대가 있나요?,그냥 내가 해결하는 게 나아. 남들한테 부담 주고 싶지도 않고.,혼자 해결하기로 했군요. 혼자서 해결하기 힘들면 주위에 의논할 사람을 찾아보세요.,NaN,NaN
1,2,청년,여성,"진로,취업,직장",해당없음,분노,노여워하는,이번 달에 또 급여가 깎였어! 물가는 오르는데 월급만 자꾸 깎이니까 너무 화가 나.,급여가 줄어 속상하시겠어요. 월급이 줄어든 것을 어떻게 보완하실 건가요?,최대한 지출을 억제해야겠어. 월급이 줄어들었으니 고정지출을 줄일 수밖에 없을 것 같아.,월급이 줄어든 만큼 소비를 줄일 계획이군요.,NaN,NaN
2,3,청년,여성,"진로,취업,직장",해당없음,분노,노여워하는,회사에 신입이 들어왔는데 말투가 거슬려. 그런 애를 매일 봐야 한다고 생각하니까 스...,회사 동료 때문에 스트레스를 많이 받는 것 같아요. 문제 해결을 위해 어떤 노력을 ...,잘 안 맞는 사람이랑 억지로 잘 지내는 것보단 조금은 거리를 두고 예의를 갖춰서 대...,스트레스받지 않기 위해선 인간관계에 있어 약간의 거리를 두는 게 좋겠군요.,NaN,NaN
3,4,청년,여성,"진로,취업,직장",해당없음,분노,노여워하는,직장에서 막내라는 이유로 나에게만 온갖 심부름을 시켜. 일도 많은 데 정말 분하고 ...,관련 없는 심부름을 모두 하게 되어서 노여우시군요. 어떤 것이 상황을 나아질 수 있...,직장 사람들과 솔직하게 이야기해보고 싶어. 일하는 데에 방해된다고.,직장 사람들과 이야기를 해 보겠다고 결심하셨군요.,NaN,NaN
4,5,청년,여성,"진로,취업,직장",해당없음,분노,노여워하는,얼마 전 입사한 신입사원이 나를 무시하는 것 같아서 너무 화가 나.,무시하는 것 같은 태도에 화가 나셨군요. 상대방의 어떤 행동이 그런 감정을 유발하는...,상사인 나에게 먼저 인사하지 않아서 매일 내가 먼저 인사한다고!,항상 먼저 인사하게 되어 화가 나셨군요. 어떻게 하면 신입사원에게 화났음을 표현할 ...,NaN,NaN


데이터 합치기

In [ ]:
# 질문과 답변만 추출
df1= df[['사람문장1','시스템문장1']]
df1.columns=['Q', 'A']
df2= df[['사람문장2','시스템문장2']]
df2.columns=['Q', 'A']
df3= df[['사람문장3','시스템문장3']]
df3.columns=['Q', 'A']

In [ ]:
# 각 추출한 질문과 답변 합치기
df4=pd.concat([df1,df2,df3],ignore_index=True)
df4

,Q,A
0,일은 왜 해도 해도 끝이 없을까? 화가 난다.,많이 힘드시겠어요. 주위에 의논할 상대가 있나요?
1,이번 달에 또 급여가 깎였어! 물가는 오르는데 월급만 자꾸 깎이니까 너무 화가 나.,급여가 줄어 속상하시겠어요. 월급이 줄어든 것을 어떻게 보완하실 건가요?
2,회사에 신입이 들어왔는데 말투가 거슬려. 그런 애를 매일 봐야 한다고 생각하니까 스...,회사 동료 때문에 스트레스를 많이 받는 것 같아요. 문제 해결을 위해 어떤 노력을 ...
3,직장에서 막내라는 이유로 나에게만 온갖 심부름을 시켜. 일도 많은 데 정말 분하고 ...,관련 없는 심부름을 모두 하게 되어서 노여우시군요. 어떤 것이 상황을 나아질 수 있...
4,얼마 전 입사한 신입사원이 나를 무시하는 것 같아서 너무 화가 나.,무시하는 것 같은 태도에 화가 나셨군요. 상대방의 어떤 행동이 그런 감정을 유발하는...
...,...,...
174808,NaN,NaN
174809,NaN,NaN
174810,NaN,NaN
174811,NaN,NaN


In [ ]:
#결측값 확인
df4.isnull().sum()

Q    10890
A    10890
dtype: int64

In [ ]:
#결측값 제거
df4= df4.dropna(axis=0)
df4.head()

,Q,A
0,일은 왜 해도 해도 끝이 없을까? 화가 난다.,많이 힘드시겠어요. 주위에 의논할 상대가 있나요?
1,이번 달에 또 급여가 깎였어! 물가는 오르는데 월급만 자꾸 깎이니까 너무 화가 나.,급여가 줄어 속상하시겠어요. 월급이 줄어든 것을 어떻게 보완하실 건가요?
2,회사에 신입이 들어왔는데 말투가 거슬려. 그런 애를 매일 봐야 한다고 생각하니까 스...,회사 동료 때문에 스트레스를 많이 받는 것 같아요. 문제 해결을 위해 어떤 노력을 ...
3,직장에서 막내라는 이유로 나에게만 온갖 심부름을 시켜. 일도 많은 데 정말 분하고 ...,관련 없는 심부름을 모두 하게 되어서 노여우시군요. 어떤 것이 상황을 나아질 수 있...
4,얼마 전 입사한 신입사원이 나를 무시하는 것 같아서 너무 화가 나.,무시하는 것 같은 태도에 화가 나셨군요. 상대방의 어떤 행동이 그런 감정을 유발하는...


In [ ]:
#결측값 없음
df4.isnull().sum()

Q    0
A    0
dtype: int64

토큰화를 위해 형태소 분석기를 사용하지 않고, 다른 방법인 학습 기반의 토크나이저를 사용할 것이다.
그래서 원 데이터에서 ?,.,!와 같은 구두점을 미리 처리해두어야하는데 구두점을 단순히 제거할수도 있지만, 여기서는 구두점 앞에 공백(띄어쓰기) 추가하여 다른 문자들과 구분하겠다.

In [ ]:
questions = []
for sentence in df4['Q']:
    # 구두점에 대해서 띄어쓰기
    # ex) 12시 땡! -> 12시 땡 !
    sentence = re.sub(r"([?.!,])", r" \1 ", sentence)
    sentence = sentence.strip()
    questions.append(sentence)

In [ ]:
answers = []
for sentence in df4['A']:
    sentence = re.sub(r"([?.!,])", r" \1 ", sentence)
    sentence = sentence.strip()
    answers.append(sentence)

In [ ]:
 tokenizer = tfds.deprecated.text.SubwordTextEncoder.build_from_corpus(
    questions + answers, target_vocab_size=2**13)

In [ ]:
print(tokenizer.subwords[:100])

[' .', ' .  ', ' ?', '수_', '이_', '어떻게_', '것_', '너무_', '을_', '게_', '는_', '도_', '할_', '내가_', '고_', '잘_', '많이_', '있을까요', '안_', '더_', '같아', '에_', '내_', '한_', '를_', '지_', '서_', '있는_', '가_', '좋은_', '하고_', '좋겠어요', '하는_', '때문에_', '하면_', '정말_', '어떤_', '해서_', '나_', '바라요', '마음이_', '기분이_', '해', '면_', '해야_', '일이_', '은_', '의_', '못_', '으로_', '것이_', '군요', '어', '로_', '만_', '다_', '친구가_', '위해_', '나', '나는_', '좋을까요', '다른_', '지금_', '상황을_', '일을_', '그_', '지', '좀_', '그런_', '나를_', '는데_', '열심히_', '거야', '화가_', '거_', '싶어', '왜_', '하시는군요', '네', '모르겠어', '때_', '요', '길_', '앞으로_', '과_', '없어', '바랄게요', '방법이_', '했어', '같아요', '지금의_', '이런_', '걸_', '사람이_', '기_', '말을_', '난_', '무슨_', '했는데_', '요즘_']


In [ ]:
print("테스트 해볼 문장: ",questions[20], '\n')
sample_string = questions[20]

#인코딩한 결과를 tokenized_string에 저장
tokenized_string=tokenizer.encode(sample_string)
print('정수 인코딩 후의 문장 : {}'.format(tokenized_string))

#이를 다시 디코딩
original_string = tokenizer.decode(tokenized_string)
print('기존문장 : {}'.format(original_string))

테스트 해볼 문장:  요즘 취업 관련해서 떠올리기만 해도 온몸이 마비될 것 같아 . 

정수 인코딩 후의 문장 : [100, 525, 7672, 4022, 279, 1273, 329, 1616, 199, 2654, 171, 7, 21, 1]
기존문장 : 요즘 취업 관련해서 떠올리기만 해도 온몸이 마비될 것 같아 .


In [ ]:
#.vocab_size를 통해 단어 집합의 크기를 확인
print('단어 집합의 크기(Vocab size) :', tokenizer.vocab_size)

단어 집합의 크기(Vocab size) : 8231


In [ ]:
START_TOKEN, END_TOKEN = [tokenizer.vocab_size], [tokenizer.vocab_size + 1]

# 시작 토큰과 종료 토큰을 고려하여 단어 집합의 크기를 + 2
VOCAB_SIZE = tokenizer.vocab_size + 2

In [ ]:
print('시작 토큰 번호 :',START_TOKEN)
print('종료 토큰 번호 :',END_TOKEN)
print('단어 집합의 크기 :',VOCAB_SIZE)

시작 토큰 번호 : [8231]
종료 토큰 번호 : [8232]
단어 집합의 크기 : 8233


In [ ]:
# 각 정수는 각 단어와 어떻게 mapping되는지 병렬로 출력
# 서브워드텍스트인코더는 의미있는 단위의 서브워드로 토크나이징한다. 띄어쓰기 단위 X 형태소 분석 단위 X
for ts in tokenized_string:
  print ('{} ----> {}'.format(ts, tokenizer.decode([ts])))

100 ----> 요즘 
525 ----> 취업 
7672 ----> 관련해서 
4022 ----> 떠올
279 ----> 리
1273 ----> 기만 
329 ----> 해도 
1616 ----> 온
199 ----> 몸이 
2654 ----> 마비
171 ----> 될 
7 ----> 것 
21 ----> 같아
1 ---->  .


In [ ]:
# 최대 길이를 100으로 정의
MAX_LENGTH = 100

# 토큰화 / 정수 인코딩 / 시작 토큰과 종료 토큰 추가 / 패딩
def tokenize_and_filter(inputs, outputs):
  tokenized_inputs, tokenized_outputs = [], []

  for (sentence1, sentence2) in zip(inputs, outputs):
    # encode(토큰화 + 정수 인코딩), 시작 토큰과 종료 토큰 추가
    sentence1 = START_TOKEN + tokenizer.encode(sentence1) + END_TOKEN
    sentence2 = START_TOKEN + tokenizer.encode(sentence2) + END_TOKEN

    tokenized_inputs.append(sentence1)
    tokenized_outputs.append(sentence2)

  # 패딩
  tokenized_inputs = tf.keras.preprocessing.sequence.pad_sequences(
      tokenized_inputs, maxlen=MAX_LENGTH, padding='post')
  tokenized_outputs = tf.keras.preprocessing.sequence.pad_sequences(
      tokenized_outputs, maxlen=MAX_LENGTH, padding='post')

  return tokenized_inputs, tokenized_outputs


In [ ]:
questions, answers = tokenize_and_filter(questions, answers)


In [ ]:
print('질문 데이터의 크기(shape) :', questions.shape)
print('답변 데이터의 크기(shape) :', answers.shape)

질문 데이터의 크기(shape) : (163922, 100)
답변 데이터의 크기(shape) : (163922, 100)


In [ ]:
print(questions[1])
print(answers[1])

[8231  340 4721  311 3947   29 8209 8160 8117 2150  448  744  492 2530
   71 2188   55  155 8209 8160 8117 6058    8   74   59    1 8232    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0]
[8231 3947   29 5827 8007  197    2 2963 5827  928  189    6  274 1942
  315  282    3 8232    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0

In [ ]:
# 텐서플로우 dataset을 이용하여 셔플(shuffle)을 수행하되, 배치 크기로 데이터를 묶는다.
# 또한 이 과정에서 교사 강요(teacher forcing)을 사용하기 위해서 디코더의 입력과 실제값 시퀀스를 구성한다.
BATCH_SIZE = 128
BUFFER_SIZE = 20000

# 디코더의 실제값 시퀀스에서는 시작 토큰을 제거해야 한다.
dataset = tf.data.Dataset.from_tensor_slices((
    {
        'inputs': questions,
        'dec_inputs': answers[:, :-1] # 디코더의 입력. 마지막 패딩 토큰이 제거된다.
    },
    {
        'outputs': answers[:, 1:]  # 맨 처음 토큰이 제거된다. 다시 말해 시작 토큰이 제거된다. 일종의 정답지라고 생각!
    },
))

dataset = dataset.cache()
dataset = dataset.shuffle(BUFFER_SIZE)
dataset = dataset.batch(BATCH_SIZE)
dataset = dataset.prefetch(tf.data.experimental.AUTOTUNE)

In [ ]:
# 임의의 샘플에 대해서 [:, :-1]과 [:, 1:]이 어떤 의미를 가지는지 테스트해본다.
print('인코더 input')
print(answers[0]) # 기존 샘플
print(answers[0].shape)
print()

print('디코더 input')
print(answers[:1][:, :-1]) # 마지막 패딩 토큰 제거하면서 길이가 99가 된다.
print(answers[:1][:, :-1].shape)
print()

print('정답지')
print(answers[:1][:, 1:]) # 맨 처음 토큰이 제거된다. 다시 말해 시작 토큰이 제거된다. 길이는 역시 99가 된다.
print(answers[:1][:, 1:].shape)

인코더 input
[8231   17  159    2  861 2516   13 4328  214    3 8232    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0]
(100,)

디코더 input
[[8231   17  159    2  861 2516   13 4328  214    3 8232    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0

# Transformer

In [ ]:
from models.transformer.transformer_model import transformer

In [ ]:
tf.keras.backend.clear_session()

# Hyper-parameters
NUM_LAYERS = 3
D_MODEL = 256
NUM_HEADS = 8
DFF = 512
DROPOUT = 0.1
#랜덤으로 0.1의 확률로 뉴런을 제거할지 말지결정. -> 과대적합 방지

model = transformer(
    vocab_size=VOCAB_SIZE,
    num_layers=NUM_LAYERS,
    dff=DFF,
    d_model=D_MODEL,
    num_heads=NUM_HEADS,
    dropout=DROPOUT)

(1, 8233, 256)
(1, 8233, 256)


In [ ]:
def loss_function(y_true, y_pred):
  y_true = tf.reshape(y_true, shape=(-1, MAX_LENGTH - 1))

  loss = tf.keras.losses.SparseCategoricalCrossentropy(
      from_logits=True, reduction='none')(y_true, y_pred)

  mask = tf.cast(tf.not_equal(y_true, 0), tf.float32)
  loss = tf.multiply(loss, mask)

  return tf.reduce_mean(loss)

In [ ]:
MAX_LENGTH = 100


optimizer = tf.keras.optimizers.Adam(beta_1=0.9, beta_2=0.98, epsilon=1e-9)

def accuracy(y_true, y_pred):
  # ensure labels have shape (batch_size, MAX_LENGTH - 1)
  y_true = tf.reshape(y_true, shape=(-1, MAX_LENGTH - 1))
  return tf.keras.metrics.sparse_categorical_accuracy(y_true, y_pred)

model.compile(optimizer=optimizer, loss=loss_function, metrics=[accuracy])


In [ ]:
EPOCHS = 30

model.fit(dataset, epochs=EPOCHS)

Epoch 1/30
1281/1281 [==============================] - 466s 341ms/step - loss: 0.5687 - accuracy: 0.0308
Epoch 2/30
1281/1281 [==============================] - 387s 302ms/step - loss: 0.4471 - accuracy: 0.0400
Epoch 3/30
1281/1281 [==============================] - 386s 302ms/step - loss: 0.4138 - accuracy: 0.0424
Epoch 4/30
1281/1281 [==============================] - 383s 299ms/step - loss: 0.3950 - accuracy: 0.0438
Epoch 5/30
1281/1281 [==============================] - 383s 299ms/step - loss: 0.3824 - accuracy: 0.0447
Epoch 6/30
1281/1281 [==============================] - 383s 299ms/step - loss: 0.3730 - accuracy: 0.0454
Epoch 7/30
1281/1281 [==============================] - 382s 298ms/step - loss: 0.3657 - accuracy: 0.0459
Epoch 8/30
1281/1281 [==============================] - 382s 298ms/step - loss: 0.3599 - accuracy: 0.0463
Epoch 9/30
1281/1281 [==============================] - 382s 298ms/step - loss: 0.3551 - accuracy: 0.0467
Epoch 10/30
1281/1281 [=======================

In [ ]:
#모델저장
model.save('tf_model.h5')

#

In [ ]:
def preprocess_sentence(sentence):
  sentence = re.sub(r"([?.!,])", r" \1 ", sentence)
  sentence = sentence.strip()
  return sentence


In [ ]:
def evaluate(sentence):
  sentence = preprocess_sentence(sentence)

  sentence = tf.expand_dims(
      START_TOKEN + tokenizer.encode(sentence) + END_TOKEN, axis=0)

  output = tf.expand_dims(START_TOKEN, 0)

  # 디코더의 예측 시작
  for i in range(MAX_LENGTH):
    predictions = model(inputs=[sentence, output], training=False)

    # 현재(마지막) 시점의 예측 단어를 받아온다.
    predictions = predictions[:, -1:, :]
    predicted_id = tf.cast(tf.argmax(predictions, axis=-1), tf.int32)

    # 만약 마지막 시점의 예측 단어가 종료 토큰이라면 예측을 중단
    if tf.equal(predicted_id, END_TOKEN[0]):
      break

    # 마지막 시점의 예측 단어를 출력에 연결한다.
    # 이는 for문을 통해서 디코더의 입력으로 사용될 예정이다.
    output = tf.concat([output, predicted_id], axis=-1)

  return tf.squeeze(output, axis=0)


def predict(sentence):
  prediction = evaluate(sentence)

  predicted_sentence = tokenizer.decode(
      [i for i in prediction if i < tokenizer.vocab_size])

  print('Input: {}'.format(sentence))
  print('Output: {}'.format(predicted_sentence))

  return predicted_sentence

In [ ]:
output = predict('너무 더워')

Input: 너무 더워
Output: 어떻게 하면 이 상황이 나아질 수 있을까요 ?


In [ ]:
output1 = predict('나 성공했어.')

Input: 나 성공했어.
Output: 다음 면접은 잘 치르고 좋은 결과가 있으면 좋겠어요 .


In [ ]:
output = predict('난 지금 기분이 좋아')

Input: 난 지금 기분이 좋아
Output: 앞으로도 좋은 일만 가득하길 바라요 .


In [ ]:
output = predict('결과가 좋아')

Input: 결과가 좋아
Output: 다음 시험을 잘 봐서 기분이 나아졌으면 좋겠어요 .


In [ ]:
output = predict('무슨일이니?')

Input: 무슨일이니?
Output: 어떻게 하면 이 상황이 나아질 수 있을까요 ?


In [ ]:
output = predict('나 오늘 합격했어')

Input: 나 오늘 합격했어
Output: 다음 시험을 잘 봐서 승진하셨으면 좋겠어요 .


In [ ]:
output = predict('오늘 헤어졌어')

Input: 오늘 헤어졌어
Output: 어떻게 하면 지금의 상황을 변화시킬 수 있을까요 ?


In [ ]:
output = predict('우리 대화가 되는거야?')

Input: 우리 대화가 되는거야?
Output: 대화를 통해 문제가 잘 해결되길 바라요 .


In [ ]:
output = predict('응원해줘')

Input: 응원해줘
Output: 앞으로도 좋은 일만 가득하길 바라요 .


In [ ]:
output = predict('시험을 망쳤어')

Input: 시험을 망쳤어
Output: 다음 시험을 앞두고 성적이 떨어져서 후회되시는군요 .  어떻게 하면 좋을까요 ?
